In [2]:
!pip install deepspeed pydantic torch transformers tensorboard


  Using cached deepspeed-0.14.3-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 128.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 159.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 160.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 105.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
# Simplified Dolly Training Script

import logging
import os
import re
from datetime import datetime
from training.consts import DEFAULT_INPUT_MODEL, SUGGESTED_INPUT_MODELS
from training.trainer import load_training_dataset, load_tokenizer
from training.generate import generate_response, load_model_tokenizer_for_generate

# Setup logging
logging.basicConfig(
    format="%(asctime)s %(levelname)s [%(name)s] %(message)s", level=logging.INFO, datefmt="%Y-%m-%d %H:%M:%S"
)
logging.getLogger("py4j").setLevel(logging.WARNING)
logging.getLogger("sh.command").setLevel(logging.ERROR)

# Configurations
input_model = "EleutherAI/pythia-2.8b"  # Change this to the desired model
gpu_family = "a10"  # Change based on your GPU type
num_gpus = 1  # Set number of GPUs available
epochs = 2
batch_size = 2
learning_rate = 5e-6

# Paths
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
model_name = "dolly"
checkpoint_dir_name = f"{model_name}__{timestamp}"
local_training_root = os.path.join(os.path.expanduser('~'), "dolly_training")
local_output_dir = os.path.join(local_training_root, checkpoint_dir_name)
tensorboard_display_dir = f"{local_output_dir}/runs"

os.makedirs(local_output_dir, exist_ok=True)

print(f"Local Output Dir: {local_output_dir}")
print(f"Tensorboard Display Dir: {tensorboard_display_dir}")

# Deepspeed configuration file path
config_file_name = f"{gpu_family}_config.json"
deepspeed_config = os.path.join(os.getcwd(), "config", config_file_name)
print(f"Deepspeed config file: {deepspeed_config}")

# Adjust batch size based on GPU type
if gpu_family == "a10":
    batch_size = 4
elif gpu_family == "a100":
    batch_size = 6

bf16_flag = "--bf16 true" if gpu_family != "v100" else "--bf16 false"

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Fake data for training (create a small dataset for the example)
fake_data_path = "fake_data.jsonl"
fake_data = [
    {"instruction": "Translate 'Hello' to French", "response": "Bonjour"},
    {"instruction": "Translate 'Goodbye' to Spanish", "response": "Adiós"}
]

with open(fake_data_path, 'w') as f:
    for entry in fake_data:
        f.write(f"{entry}\n")

# Training command
training_command = f"""
deepspeed --num_gpus={num_gpus} \
    --module training.trainer \
    --input-model {input_model} \
    --deepspeed {deepspeed_config} \
    --epochs {epochs} \
    --local-output-dir {local_output_dir} \
    --per-device-train-batch-size {batch_size} \
    --per-device-eval-batch-size {batch_size} \
    --logging-steps 10 \
    --save-steps 200 \
    --save-total-limit 20 \
    --eval-steps 50 \
    --warmup-steps 50 \
    --test-size 200 \
    --lr {learning_rate} \
    {bf16_flag} \
    --data_path {fake_data_path}
"""

print(training_command)

# Run training (uncomment the following line to execute)
# !{training_command}

# Load model and tokenizer for inference
model, tokenizer = load_model_tokenizer_for_generate(local_output_dir)

# Example instructions
instructions = [
    "Write a love letter to Edgar Allan Poe.",
    "Write a tweet announcing Dolly, a large language model from Databricks.",
    "I'm selling my Nikon D-750, write a short blurb for my ad.",
    "Explain to me the difference between nuclear fission and fusion.",
    "Give me a list of 5 science fiction books I should read next."
]

# Set additional pipeline args based on GPU type
pipeline_kwargs = {'torch_dtype': "auto"}
if gpu_family == "v100":
    pipeline_kwargs['torch_dtype'] = "float16"
elif gpu_family in ["a10", "a100"]:
    pipeline_kwargs['torch_dtype'] = "bfloat16"

# Generate responses
for instruction in instructions:
    response = generate_response(instruction, model=model, tokenizer=tokenizer, **pipeline_kwargs)
    if response:
        print(f"Instruction: {instruction}\n\n{response}\n\n-----------\n")


ModuleNotFoundError: No module named 'click'